In [1]:
# import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd 

In [2]:
def LinkLocalidad(provincia=str):
    """ Esta función recibe una provincia y devuelve los links de cada localidad de la provincia """
    if provincia != '':
        localidadesUrls = []
        url = f'https://codigo-postal.co/argentina/{provincia}'
        response = requests.get(url)
        if response.status_code == 200:
            contador = 0
            soup = BeautifulSoup(response.content, 'html.parser')
            localidades = soup.find('ul',{'class': 'cities'})
            for urls in localidades.find_all('a'):
                contador += 1
                localidadesUrls.append({'provincia':provincia,'idlocalidad': contador,'localidad': urls.text,'url': urls['href']})
    return localidadesUrls

In [3]:
listaLocalidades = LinkLocalidad('salta')

In [41]:
listaLocalidades[:3]

[{'provincia': 'salta',
  'idlocalidad': 1,
  'localidad': '20 de Febrero',
  'url': 'https://codigo-postal.co/argentina/salta/20-de-febrero/'},
 {'provincia': 'salta',
  'idlocalidad': 2,
  'localidad': 'Ablome',
  'url': 'https://codigo-postal.co/argentina/salta/ablome/'},
 {'provincia': 'salta',
  'idlocalidad': 3,
  'localidad': 'Abra de Araguyoc',
  'url': 'https://codigo-postal.co/argentina/salta/abra-de-araguyoc/'}]

In [5]:
def CpaLinks(lista=list):
    """ Esta función recibe una lista y devuelve los cpa de las localidades junto con los links de las calles para encontrar el cpa """
    linkCalles = []
    localidadesCpa = []
    for elemento in lista:
        response = requests.get(elemento['url'])
        soup = BeautifulSoup(response.content, 'html.parser')
        localidades =soup.find_all('tbody')
        for localidad in localidades:
            tr = localidad.find_all('tr')
            url= localidad.find_all('a')
            for index, data in enumerate(url):
                if 'Buscar CPA' in data:
                    linkCalles.append({'provincia': tr[index].find_all('td')[0].text,'idlocalidad':elemento['idlocalidad'],'localidad': tr[index].find_all('td')[1].text,'linkCalles':data['href']})
                else: 
                    localidadesCpa.append({'provincia': tr[index].find_all('td')[0].text,'idlocalidad':elemento['idlocalidad'],'localidad': tr[index].find_all('td')[1].text, 'pc': tr[index].find_all('td')[2].text, 'cpa':data.text})
    return  localidadesCpa, linkCalles

In [17]:
listaLocalidades1 = listaLocalidades[:3]

In [18]:
data , linkCalles =  CpaLinks(listaLocalidades1)

In [42]:
localidad = pd.DataFrame(data)

In [43]:
localidad

,provincia,idlocalidad,localidad,pc,cpa
0,Salta,1,20 de Febrero,4425,A4425XBN
1,Salta,1,Acosta,4425,A4425XAA
2,Salta,1,Amblayo,4425,A4425XBQ
3,Salta,1,Bodeguita,4425,A4425XAC
4,Salta,1,Campos de Alemania,4425,A4425XAB
...,...,...,...,...,...
124,Salta,3,Uchuyoc,4633,A4633XAX
125,Salta,3,Valle Delgado,4633,A4633XAY
126,Salta,3,Villa Alem,4633,A4633XAT
127,Jujuy,3,Volcan Higuera,4633,Y4633XAB


In [23]:
linkCalles

[{'provincia': 'Salta',
  'idlocalidad': 1,
  'localidad': 'Guachipas',
  'linkCalles': 'https://codigo-postal.co/argentina/salta/calles-de-guachipas/'},
 {'provincia': 'Salta',
  'idlocalidad': 2,
  'localidad': 'El Carril',
  'linkCalles': 'https://codigo-postal.co/argentina/salta/calles-de-el-carril/'},
 {'provincia': 'Salta',
  'idlocalidad': 3,
  'localidad': 'Iruya',
  'linkCalles': 'https://codigo-postal.co/argentina/salta/calles-de-iruya/'}]

In [29]:
def LinkCalle(lista=list):
    """ Esta funcion recibe una lista y devuelve las urls de la calles"""
    contador = 0
    urlsCalle = []
    for i in lista:
        response = requests.get(i['linkCalles'])
        soup = BeautifulSoup(response.content, 'html.parser')
        calles =soup.find('ul',{'class':"three_columns"})
        for j in calles.find_all('a'):
            contador += 1
            urlsCalle.append({'provincia': i['provincia'],'idlocalidad': i['idlocalidad'],'idcalle':contador,'nomnrecalle':j.text,'link':j['href']})
    return urlsCalle

In [30]:
urlCalles = LinkCalle(linkCalles)

In [44]:
urlCalles[:5]

[{'provincia': 'Salta',
  'idlocalidad': 1,
  'idcalle': 1,
  'nomnrecalle': 'Calle Carlos Gomez Alzaga',
  'link': 'https://codigo-postal.co/argentina/salta/guachipas/carlos-gomez-alzaga/'},
 {'provincia': 'Salta',
  'idlocalidad': 1,
  'idcalle': 2,
  'nomnrecalle': 'Calle Dr Bernardo Frias',
  'link': 'https://codigo-postal.co/argentina/salta/guachipas/dr-bernardo-frias/'},
 {'provincia': 'Salta',
  'idlocalidad': 1,
  'idcalle': 3,
  'nomnrecalle': 'Calle Dr Eugenio Romanou',
  'link': 'https://codigo-postal.co/argentina/salta/guachipas/dr-eugenio-romanou/'},
 {'provincia': 'Salta',
  'idlocalidad': 1,
  'idcalle': 4,
  'nomnrecalle': 'Calle Dr Indalecio Gomez',
  'link': 'https://codigo-postal.co/argentina/salta/guachipas/dr-indalecio-gomez/'},
 {'provincia': 'Salta',
  'idlocalidad': 1,
  'idcalle': 5,
  'nomnrecalle': 'Calle Gral A Saravia',
  'link': 'https://codigo-postal.co/argentina/salta/guachipas/gral-a-saravia/'}]

In [37]:
def CallePca(lista=list):
    """ Esta funcion recibe una lista y devuelve los cpa de cada calle"""
    cpa = []
    for i in lista:
        response = requests.get(i['link'])
        soup = BeautifulSoup(response.content, 'html.parser')
        tabla = soup.find_all('tbody')
        for j in tabla:
            tr = j.find_all('tr')
            for elemento in tr:
                cpa.append({'provincia': i['provincia'],'idlocalidad': i['idlocalidad'],'idcalle':i['idcalle'],'calle':elemento.find_all('td')[0].text,'cp':elemento.find_all('td')[4].text, 'cpa':elemento['data-cpa'],'desde':elemento['data-desde'], 'hasta':elemento['data-hasta'],'aplica a':elemento.find_all('td')[3].text})
    return cpa

In [38]:
pca = CallePca(urlCalles)

In [45]:
pca[:5]

[{'provincia': 'Salta',
  'idlocalidad': 2,
  'idcalle': 13,
  'calle': 'Calle 1 de Mayo',
  'cp': '4421',
  'cpa': 'A4421BIO',
  'desde': '0',
  'hasta': '0',
  'aplica a': 'números pares'},
 {'provincia': 'Salta',
  'idlocalidad': 2,
  'idcalle': 13,
  'calle': 'Calle 1 de Mayo',
  'cp': '4421',
  'cpa': 'A4421AAA',
  'desde': '1',
  'hasta': '99',
  'aplica a': 'números impares'},
 {'provincia': 'Salta',
  'idlocalidad': 2,
  'idcalle': 13,
  'calle': 'Calle 1 de Mayo',
  'cp': '4421',
  'cpa': 'A4421AAB',
  'desde': '2',
  'hasta': '100',
  'aplica a': 'números pares'},
 {'provincia': 'Salta',
  'idlocalidad': 2,
  'idcalle': 13,
  'calle': 'Calle 1 de Mayo',
  'cp': '4421',
  'cpa': 'A4421AAC',
  'desde': '101',
  'hasta': '199',
  'aplica a': 'números impares'},
 {'provincia': 'Salta',
  'idlocalidad': 2,
  'idcalle': 13,
  'calle': 'Calle 1 de Mayo',
  'cp': '4421',
  'cpa': 'A4421AAD',
  'desde': '102',
  'hasta': '200',
  'aplica a': 'números pares'}]